In [68]:
import torch
from torchvision import models
import torch.nn as nn
import torch.nn.functional as F
from collections import OrderedDict

class modified_efficient(nn.Module):
    def __init__(self, model):
        super(modified_efficient, self).__init__()
        self.use_cuda = torch.cuda.is_available()
        self.device = torch.device("cuda" if self.use_cuda else "cpu")
        if self.use_cuda:
            torch.cuda.manual_seed_all(1024)
        self.model = model
        self.model.to(self.device)

        random.seed(1024)
        torch.manual_seed(1024)

    def forward(self, x):
        outputs = []

        def hook(module, input, output):
            outputs.append(output)

        self.model.features[0][1].register_forward_hook(hook)
        self.model.features[1][0].register_forward_hook(hook)
        self.model.features[2][1].register_forward_hook(hook)
        self.model.features[3][1].register_forward_hook(hook)
        self.model.features[4][2].register_forward_hook(hook)
        self.model.features[5][2].register_forward_hook(hook)
        self.model.features[6][3].register_forward_hook(hook)
        self.model.features[7][0].register_forward_hook(hook)
        self.model.features[8][1].register_forward_hook(hook)

        with torch.no_grad():
            eff_result = self.model(x.to(self.device))

        embedding_vectors_1 = outputs[0].cpu().detach()
        embedding_vectors_2 = outputs[1].cpu().detach()
        embedding_vectors_3 = outputs[2].cpu().detach()
        embedding_vectors_4 = outputs[3].cpu().detach()
        embedding_vectors_5 = outputs[4].cpu().detach()
        embedding_vectors_6 = outputs[5].cpu().detach()
        embedding_vectors_7 = outputs[6].cpu().detach()
        embedding_vectors_8 = outputs[7].cpu().detach()
        embedding_vectors_9 = outputs[8].cpu().detach()

        return eff_result, embedding_vectors_1, embedding_vectors_2, embedding_vectors_3,embedding_vectors_4,embedding_vectors_5,embedding_vectors_6,embedding_vectors_7,embedding_vectors_8,embedding_vectors_9


model = models.efficientnet_b0(pretrained=True)
x = torch.randn(1, 3, 224, 224)
net = modified_efficient(model)
eff_result, embedding_vectors_1, embedding_vectors_2, embedding_vectors_3,embedding_vectors_4,embedding_vectors_5,embedding_vectors_6,embedding_vectors_7,embedding_vectors_8,embedding_vectors_9 = net(x)


In [69]:
# Export the model
torch.onnx.export(net,               
                  x,                   
                  "efficientnet.onnx",
                  export_params=True,  
                  opset_version=18,    
                  do_constant_folding=True,  
                  input_names=['input'],   
                  output_names=["output",'output_1', 'output_2', 'output_3','output_4', 'output_5', 'output_6','output_7', 'output_8', 'output_9'],  
                 )

============== Diagnostic Run torch.onnx.export version 2.0.1+cpu ==============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

